# Instalations

In [1]:
# !pip install psgtray
# !pip install pysimplegui
# !pip install keyboard

# Imports

In [2]:
from threading import Thread
import PySimpleGUI as sg
from psgtray import SystemTray
import keyboard
from datetime import datetime

# Hotkeys

In [3]:
def hotkey_exec(window, i):
    window.write_event_value('-HOTKEY-', f'{i}')
    
def watch_keyboard(window):
    for i in range(0,10):
        keyboard.add_hotkey(f'ctrl + shift + {i}', hotkey_exec, args =(window, i))

def reset_keyboard():
    for i in range(0,10):
            keyboard.remove_hotkey(f'ctrl + shift + {i}')

# Snippets Func

In [4]:
add_snippet = lambda key, text: keyboard.add_abbreviation(key, text)
rm_snippet = lambda key: keyboard.remove_abbreviation(e)

# Actions

In [5]:
def exec_script(script:str):
    pass

def exec_hotkeys(keys:str):
    pass

def exec_system_termina(cmd:str):
    pass

def activate_snippets():
    pass

def deactivate_snippets():
    pass



# Mode

In [6]:
class Mode(object):
    def __init__(self, 
                 name:str
                ):
        self.__name = name

    def __str__(self):
        return self.__name


# GUI

In [14]:
# Gerando elemento de cada botão
def gen_key_comp(i:int):
    i = i + 1 if i < 9 else 0 
    return sg.Frame(
        f'FUNC-{i}',
        [
        [sg.T(f'Flavor text of Func {i}', key=f'-DESCRIPTION{i}-', s=(15,2))],
        [sg.Image(sg.EMOJI_BASE64_HAPPY_THUMBS_UP, size=(100, 100), key=f'-THUMB-{i}')],
        [sg.Button('Edit', key=f'-EDIT{i}-', expand_x=True)]
        ]
    )

# Gerando linhas de botões
gen_key_lines = lambda start, end: [gen_key_comp(i) for i in range(start, end)]
        


def main():
    
    all_modes = ['Inativo', 'Stream Deck', 'Rodrigo Faro', 'Devil Man']
    
    # Estado da aplicação
    cur_mode = 0
    selected_mode = 0
    in_selection = False
    time_last_comand = datetime.now() # Iniciando em uma data específica
    
    
    menu = [f'MODO: {cur_mode}', ['Show Window', 'Hide Window', '---', 'Exit']]
    tooltip = lambda x = cur_mode: f'Modo: {x}'

    layout = [
        gen_key_lines(0,5),
        gen_key_lines(5,10),
        [sg.Button('Exit', expand_x=True)]
    ]

    window = sg.Window('PyHotKeys', layout, finalize=True, enable_close_attempted_event=True)
    watch_keyboard(window)
    tray = SystemTray(menu, single_click_events=False, window=window, tooltip=tooltip(), icon=sg.EMOJI_BASE64_HAPPY_JOY)
    
    
    def show_message(text:str):
        nonlocal tray 
        tray.show_message('PyHotKeys', text)
        
        
    def mode_navigate(add:bool):
        nonlocal all_modes, cur_mode, selected_mode
        len_modes = len(all_modes)
        
        selected_mode = selected_mode + 1 if add else selected_mode + 1
        selected_mode = 0  if len_modes < selected_mode + 1 else selected_mode
        selected_mode = len_modes - 1  if selected_mode < 0 else selected_mode
        
        
        show_message(f"ATUAL: {all_modes[cur_mode]}\nSELECIONADO: {all_modes[selected_mode]}")
    
    
    show_message(f'Iniciado, modo no modo: {cur_mode}')
        
    
    while True:
        event, values = window.read()

        if event == tray.key:
            event = values[event]
        if event in (sg.WIN_CLOSED, 'Exit'):
            break
        if event == '-HOTKEY-':
            cur_time = datetime.now()
            diff_last_time = (cur_time - time_last_comand).total_seconds()
            if  diff_last_time > 1: # Bloqueando comandos com menos de 1 segundo de tempo
                time_last_comand = cur_time
                
                
                if values[event] == '0':
                    if in_selection:
                        in_selection = False
                        cur_mode = selected_mode
                        show_message(f"Selecionado modo: {all_modes[cur_mode]}")
                        
                    else:
                        in_selection = True
                        show_message(f"Entrando em modo de seleção.\nAtualmente em: {all_modes[cur_mode]}")

                
                if values[event] == '8':
                    if in_selection == True:
                        mode_navigate(False)
                    
                if values[event] == '9':
                    if in_selection == True:
                        mode_navigate(True)
                        
        
        if event in ('Show Window', sg.EVENT_SYSTEM_TRAY_ICON_DOUBLE_CLICKED):
            window.un_hide()
            window.bring_to_front()
        elif event in ('Hide Window', sg.WIN_CLOSE_ATTEMPTED_EVENT):
            window.hide()
            tray.show_icon()
        

    tray.close()            
    window.close()
    reset_keyboard()
    
if __name__ == '__main__':
    main()

2